# 練習網址：http://www.chinabooktrading.com/song/
## 目標
爬取全部宋詞之作者名、押韻、體詩、及詩等。
## 步驟
1. 使用selenium登入會員，得cookies。
2. 將所得cookies塞入requests內，方便後續爬蟲。
3. 因所爬取內容由javascript寫成，故尋找關鍵網址後，開始爬取資料。

In [34]:
#!/usr/bin/env python
#coding=utf-8

In [35]:
# 建立各種 HTTP 請求，從網頁伺服器上取得想要的資料
import requests

# 用來解析處理requests取得的數據; processing XML and HTML
#from lxml import html

# 一個HTML解析器，可以處理各種奇怪的HTML
from bs4 import BeautifulSoup as bs

# 睡眠用
from time import sleep

# 處理javascript用
import json

# 能處理載入、整理與視覺化等常見的資料應用套件
import pandas as pd

# 隨機睡眠用
import random

import os

print('套件載入完成')

套件載入完成


### 設定requests

In [36]:
s = requests.Session()

# 得cookies
# 發現從左側宋詞列表按右鍵，"複製連結“得超連結網址
res = s.get('http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u=1&b=4')
cookies = s.cookies.get_dict()
# print(s.cookies.get_dict())

In [37]:
# 解析網頁
html = bs(res.text)
print(html)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="zh-CN" xml:lang="zh-CN" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="poem" http-equiv="Description"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>poem</title>
<link href="static/fulltext.css" rel="stylesheet" type="text/css"/>
<script type="text/javascript">
			var isGlVisible = false;
			var $ = function(id) {
				return document.getElementById(id);
			};
			var switchGl = function(id) {
				if (isGlVisible == true) {
					isGlVisible = false;
					if (id !== "btngl3") {
						$("btngl1").innerHTML = "顯示格律";
						$("btngl2").innerHTML = "顯示格律";
					} else {
						$("btngl3").innerHTML = "顯示格律";
					}
					var elems = document.getElementsByTagName("p");
					for (var i=0; i<elems.length; i++) {
						if (elems[i].className == "gl") {
							elems[i].style.display = "none";
						}
					}
				} else {
				

In [38]:
# 縮小範圍
html = html.find("div")
html

<div class="content">
<div class="content">
<h2 class="nametitle subtitle TEXT_MEDIUM">詩:宋詩  日詩   (宋太祖)</h2>
<div class="viewopts">
                            
                                【<a class="" href="javascript://;" id="btngl3" onclick="switchGl('btngl3');">顯示格律</a>】
                                【<a class="" href="resultList.jsp?e=1&amp;a=1&amp;b=1&amp;v=7&amp;x=00&amp;s=%E6%9C%88" target="sider">查與本詩形式相似詩</a>】
                            
                        </div>
<div style="clear: both; margin-bottom: 25px;">
<h3 class="TEXT_DARK"> 日詩   </h3>
<h4 class="TEXT_LIGHT">宋太祖</h4>
<h4 class="TEXT_LIGHT">押六月</h4>
<h4 class="TEXT_LIGHT">古體詩</h4>
<p>欲出未出光辣達，千山萬山如火發。</p>
<p class="gl">仄仄仄仄平仄仄，平平仄平○仄仄。</p>
<p>須臾走向天上來，逐却殘星趕却月。</p>
<p class="gl">平平仄仄平仄平，仄仄平平？仄仄。</p>
</div>
<h2 class="subtitle TEXT_MEDIUM">帶註釋文本</h2>
<div>
<h3 class="TEXT_DARK"> 日詩   </h3>
<p>欲出未出光辣達，<span class="comment">《庚溪詩話》作太陽初出光赫赫</span>千山萬山如火發。須臾走向天上來，<span class="comment">《庚溪詩話》作一輪頃刻上天衢</span>逐却殘星趕却<spa

### 爬蟲

#### 1. 風險規避，定時存擋

In [42]:
## 風險規避，定時存檔
def save():
    print("-" * 80)
    print("**定時存檔中...") # 檢查哨
    dn = "ouptput" # 設立output資料夾
    file = "SongShi_content_" + str(i) # 宋詞檔名 # i == 資料筆數
    error_file = "error_list" # 錯誤連結清單檔名

    try:
        if not os.path.exists(dn):   
            os.makedirs(dn)
    except:
        pass

    # 存json格式
    ## 宋詞
    f = open(dn + "/"+ file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(final_dic, f) # 寫入json格式
    f.close() # 關閉路徑
    
    ## 錯誤清單
    e = open(dn + "/"+ error_file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(error_list, e) # 寫入json格式
    e.close() # 關閉路徑
    print("json存檔完成") # 檢查哨

    # 存csv格式
    ## 宋詞
    final_dic_df = pd.DataFrame(final_dic)
    final_dic_df.to_csv(dn + "/"+ file + ".csv",encoding="utf-8", index=False)
    
    ## 錯誤清單
    error_list_df = pd.DataFrame(error_list)
    error_list_df.to_csv(dn + "/"+ error_file + ".csv",encoding="utf-8", index=False)
    print("csv存檔完成") # 檢查哨
    print("-" * 80)
    print("-" * 80)

    # DONE
    print("存檔完成") # 檢查哨

#### 2. 爬蟲

In [43]:
# 生成隨機user-agent
UA_pool = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
    'Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)',
    'Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.6.37 Version/11.00',
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0.1) Gecko/20121011 Firefox/16.0.1',
    'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25',
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52"
]

# 隨機生成UA的函式
def UA():
    num = random.randint(0, 21) 
    UA = UA_pool[num]
    headers = {'User-Agent':UA}
    return headers

In [ ]:
# 設個最終字串，供爬完後放入
final_dic = [] # 總字典
error_list = [] #錯誤回報

for i in range(1,254240+1):

    URL = 'http://www.chinabooktrading.com/song/fullText.jsp?a=1&e=1&u='+str(i)+'&b=4'
    print("抓取網址: ", URL) # 檢查哨
    headers = UA() # 代理ip
    print("偽user-agent: ", headers)

    # 發現從左側宋詞列表按右鍵，"複製連結“得超連結網址
    res = s.get(URL, headers = headers) # 詩連結
    
    try:
        # 解析網頁
        html = bs(res.text)
    
        # 取詞體
        poem  = html.find("div", style = "clear: both; margin-bottom: 25px;")
        print("解析網頁...") # 檢查哨

        # 要存入的內容
        title = poem.find("h3", class_ = "TEXT_DARK").text # 詞名
        author = poem.find_all("h4", class_ = "TEXT_LIGHT")[0].text # 作者名
        rhyme = poem.find_all("h4", class_ = "TEXT_LIGHT")[1].text # 押韻
        poemType = poem.find_all("h4", class_ = "TEXT_LIGHT")[2].text # 體詩

        # 詩內容
        content =  poem.find_all("p", class_ = "")
        # 將詩list合併成字串
        text = ""  # str
        for x in range (len(content)):
            text += content[x].text

        poemCount = html.find_all("h4", class_ = "TEXT_LIGHT")[-1].text # 作者總共做了幾首詩
        history = html.find_all("p")[-1].text # 作者的生平故事
        print("id =", i, ", 「", title, "」參數設定完成") # 檢查哨


        # 創詞字典
        dic= {
            "id": i,
            "詞名": title,
            "作者": author,
            "押韻": rhyme,
            "體詩": poemType,
            "內容": text,
            "作者共做了幾首詩": poemCount,
            "作者生平": history,
            "網址": URL
             }

        # 加入總字典
        final_dic.append(dic)
        print("加入總字典完成") # 檢查哨
        print("-" * 80)

        # 隨機秒數休眠
        #sleepTime = random.randint(1, 3) 
        #sleep(sleepTime)
        
        if i % 50000 ==0: # 每5萬筆存一次
            save()
            
        
    except:
        # 將錯誤連結網址建成字典
        error_dic = {
            "id":i,
            "網址":URL
        }
        print("***錯誤連結id: ", i, ", 網址: ", URL)
        
        # 將「錯誤連結字典」放入「錯誤連結清單」
        error_list.append(error_dic )
        print("***錯誤連結加入錯誤清單完成")
        print("-" * 80)
        
print("爬蟲Done!")